In [6]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [8]:
SQuAD = pd.read_json("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
SQuAD.head()

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [10]:
# SQuAD.head(1).values

In [14]:
# del  SQuAD['version']
cols = ['text', 'question', 'answer']

comp_list = []
for index, row in SQuAD.iterrows():
    for i in range(len(row['data']['paragraphs'])):
        for j in (row['data']['paragraphs'][i]['qas']):
            temp_list = []
            temp_list.append(row["data"]["paragraphs"][i]["context"])
            temp_list.append(j["question"])
            if j["answers"]:
                temp_list.append(j["answers"][0]["text"])
            else:
                temp_list.append("")
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list,columns=cols)

In [17]:
new_df.to_csv("SQuAD_data.csv", index=False)
data = pd.read_csv("SQuAD_data.csv")
data.head()

,text,question,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What was the name of Beyoncé's first solo album?,Dangerously in Love
1,Following the disbandment of Destiny's Child i...,What is the name of Beyoncé's alter-ego?,Sasha Fierce
2,"A self-described ""modern-day feminist"", Beyonc...",What magazine named Beyoncé as the most powerf...,Forbes
3,"Beyoncé Giselle Knowles was born in Houston, T...",Beyoncé was raised in what religion?,Methodist
4,Beyoncé attended St. Mary's Elementary School ...,What choir did Beyoncé sing in for two years?,St. John's United Methodist Church


In [18]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [19]:
random_num = np.random.randint(0, len(data))
question = data["question"][random_num]
text = data["text"][random_num]

In [22]:
!mkdir data
!mv SQuAD_data.csv data/

mkdir: cannot create directory ‘data’: File exists


In [23]:
question

'What type of instrument is the gamelan?'

In [24]:
text

'Electric instruments such as the electric guitar, the electric bass and the ondes Martenot appear occasionally in the classical music of the 20th and 21st centuries. Both classical and popular musicians have experimented in recent decades with electronic instruments such as the synthesizer, electric and digital techniques such as the use of sampled or computer-generated sounds, and instruments from other cultures such as the gamelan.'

In [25]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 88 tokens.


In [27]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

for token, id in zip(tokens, input_ids):
    print("{:8}{:8}".format(token, id))

[CLS]        101
what        2054
type        2828
of          1997
instrument    6602
is          2003
the         1996
game        2208
##lan       5802
?           1029
[SEP]        102
electric    3751
instruments    5693
such        2107
as          2004
the         1996
electric    3751
guitar      2858
,           1010
the         1996
electric    3751
bass        3321
and         1998
the         1996
on          2006
##des       6155
mart       20481
##eno      16515
##t         2102
appear      3711
occasionally    5681
in          1999
the         1996
classical    4556
music       2189
of          1997
the         1996
20th        3983
and         1998
21st        7398
centuries    4693
.           1012
both        2119
classical    4556
and         1998
popular     2759
musicians    5389
have        2031
experimented   20918
in          1999
recent      3522
decades     5109
with        2007
electronic    4816
instruments    5693
such        2107
as          2004
the      

In [28]:
# first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print(sep_idx)

# number of tokens in segment A - question
num_seg_a = sep_idx + 1
print(num_seg_a)

#number of tokens in segment B - text
num_seg_b = len(input_ids) - num_seg_a
print(num_seg_b)

segment_ids = [0]*num_seg_a + [1]*num_seg_b
print(segment_ids)

assert len(segment_ids) == len(input_ids)

10
11
77
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [29]:
output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
print(output.start_logits, output.end_logits)

tensor([[ -6.3336,  -6.1691,  -8.1008,  -7.9677,  -8.8729,  -8.8805,  -8.5801,
          -8.5550, -10.0506,  -9.7406,  -6.3336,   3.5561,  -3.8884,  -6.2494,
          -6.7600,  -1.4761,   0.4539,  -3.7079,  -7.5307,  -2.8088,  -1.5048,
          -3.9993,  -8.1322,  -3.3581,  -2.5602,  -6.2752,  -6.7095,  -7.3740,
          -6.1552,  -5.7342,  -4.6686,  -7.0581,  -6.6008,  -3.5871,  -4.6654,
          -8.2460,  -7.2821,  -6.7642,  -8.6052,  -6.8894,  -7.3000,  -7.0785,
          -3.1269,  -2.9306,  -6.7148,  -1.3822,  -3.9576,  -5.5832,  -1.1957,
          -6.5481,  -4.3572,  -5.8244,  -3.6150,   6.8833,  -2.8930,  -5.3856,
          -5.2018,   0.7933,   1.3829,  -6.9217,  -1.4971,  -6.9072,  -0.9392,
          -4.8722,  -7.5233,  -8.1356,  -5.9448,  -4.8833,  -7.6410,  -3.4765,
          -7.8437,  -2.1763,  -7.5898,  -6.3052,  -5.2956,  -7.5174,  -5.6796,
           0.8051,  -3.8917,  -3.4086,  -3.0935,  -6.3545,  -6.5248,  -1.0897,
          -1.7854,  -6.1909,  -6.8574,  -6.3336]], g

In [32]:
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
print (answer_start, answer_end)

if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end + 1])
else:
    print("I am unable to answer your question? Please pardon")

print("Text: \n{}".format(text.capitalize()))
print("\nQuestion: \n{}".format(question.capitalize()))
print("\nAnswer: \n{}".format(answer.capitalize()))

tensor(53) tensor(53)
Text: 
Electric instruments such as the electric guitar, the electric bass and the ondes martenot appear occasionally in the classical music of the 20th and 21st centuries. both classical and popular musicians have experimented in recent decades with electronic instruments such as the synthesizer, electric and digital techniques such as the use of sampled or computer-generated sounds, and instruments from other cultures such as the gamelan.

Question: 
What type of instrument is the gamelan?

Answer: 
Electronic


In [35]:
answer = tokens[answer_start]

for i in range(answer_start+1, answer_end):
    if tokens[i][0:2] == "##":
        answer +=tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [ ]:
def question_answer(question, text):

    # tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)

    # string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # segment IDs
    # first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    # number of tokens in segment A - qustion 
    num_seg_a = sep_idx + 1

    # number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a

    # list of 0s and 1s
    segment_ids =[0]*num_seg_a + [1]*num_seg_b

    assert len(segment_ids) == len(input_ids)

    # model output using input ids and sedment ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    # reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    if answer_end >= answer_start:
        